In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from make_rankings import *
import os.path

# IMPORTING SPEEDRUN DATA

When importing, "duplicate" runs (ie multiple runs on the same quest, by the same runner, using the same weapon) are eliminated, so that only the fastest of the duplicates remains

In [ ]:
#Check for existence of freestyle.csv and ta.csv
if os.path.exists('freestyle.csv') and os.path.exists('ta.csv'):
    freestyle = pd.read_csv('freestyle.csv')
    ta = pd.read_csv('ta.csv')

else:
    #Get Freestyle and TA rankings from 'speedrun_data.csv'
    freestyle,ta = make_rankings('speedrun_data.csv')

In [ ]:
#Prepare figure properties for later graphs
figure_size = (16,7)
font_size = 20

# 1. - INITIAL ANALYSIS
### Check the average clear time per weapon

In [ ]:
#Very early analysis: just get average TOP clear time by weapons/monsters
avg_freestyle = average_top_runs(freestyle,'Monster/Weapon')
avg_ta = average_top_runs(ta,'Monster/Weapon')

#Plot average freestyle times
sns.set_context('notebook')
sns.set_style('dark')

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=avg_freestyle).set_title('Average clear time by Monster - Freestyle',fontsize=font_size)

#Plot average TA times
plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=avg_ta).set_title('Average clear time by Monster - TA',fontsize=font_size)

In [ ]:
#Same analysis, but considering top times by weapon/QUEST
avg_freestyle = average_top_runs(freestyle,'Quest/Weapon')
avg_ta = average_top_runs(ta,'Quest/Weapon')

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=avg_freestyle).set_title('Average clear time by Quest - Freestyle',fontsize=font_size)

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=avg_ta).set_title('Average clear time by Quest - TA Rules',fontsize=font_size)

# 2. - SECOND ANALYSIS
### Check the average clear time of *only the fastest runs* per weapon

In [ ]:
#Second analysis: filter only the top runs and see the average times
fs_all_weapons = filter_by_weapon(freestyle,filter_by='Monster Name')
ta_all_weapons = filter_by_weapon(ta,filter_by='Monster Name')

#Get average times
fs_aw_avg = average_top_runs(fs_all_weapons,'Monster/Weapon')
ta_aw_avg = average_top_runs(ta_all_weapons,'Monster/Weapon')

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=fs_aw_avg).set_title('Average TOP clear time by Monster - Freestyle', fontsize =font_size)

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=ta_aw_avg).set_title('Average TOP clear time by Monster - TA', fontsize =font_size)

In [ ]:
#Create 'Weapon (long)' column to have long form weapon names
#First, get the inverse weapon_dict to go from short names to long names
inv_weapon_dict = {value: key for key, value in weapon_dict.items()}

#Apply mapping from inv_weapon-dict
fs_aw_avg['Weapon (long)']=fs_aw_avg['Weapon'].apply(lambda x: inv_weapon_dict[x])
ta_aw_avg['Weapon (long)']=ta_aw_avg['Weapon'].apply(lambda x: inv_weapon_dict[x])

#Numerical values
print('Freestyle average top clear times per weapon (1 entry per monster)')
print(fs_aw_avg[['Weapon (long)','Time (s)']])
print('\nTA Rules average top clear times per weapon (1 entry per monster)')
print(ta_aw_avg[['Weapon (long)','Time (s)']])

In [ ]:
#Now repeat second analysis, but separate entries by Quest rather than Monster

fs_all_weapons = filter_by_weapon(freestyle,filter_by='Quest')
ta_all_weapons = filter_by_weapon(ta,filter_by='Quest')

fs_aw_avg_q = average_top_runs(fs_all_weapons,'Quest/Weapon')
ta_aw_avg_q = average_top_runs(ta_all_weapons,'Quest/Weapon')

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=fs_aw_avg).set_title('Average TOP clear time by Monster - Freestyle', fontsize =font_size)

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=ta_aw_avg).set_title('Average TOP clear time by Monster - TA', fontsize =font_size)

In [ ]:
#Create 'Weapon (long)' column to have long form weapon names
#First, get the inverse weapon_dict to go from short names to long names
inv_weapon_dict = {value: key for key, value in weapon_dict.items()}

#Apply mapping from inv_weapon-dict
fs_aw_avg_q['Weapon (long)']=fs_aw_avg_q['Weapon'].apply(lambda x: inv_weapon_dict[x])
ta_aw_avg_q['Weapon (long)']=ta_aw_avg_q['Weapon'].apply(lambda x: inv_weapon_dict[x])

#Numerical values
print('Freestyle average top clear times per weapon (1 entry per Quest)')
print(fs_aw_avg_q[['Weapon (long)','Time (s)']])
print('\nTA Rules average top clear times per weapon (1 entry per Quest)')
print(ta_aw_avg_q[['Weapon (long)','Time (s)']])

## 2.1 - TIER LISTS BASED ON SECOND ANALYSIS

Weapons will be classified in 'n_tiers' tiers, as chosen by the user.

In [ ]:
#Make some tier lists

#Freestyle
tl_fs_quest   = make_tiers(fs_aw_avg_q,n_tiers=7) #By quest
tl_fs_monster = make_tiers(fs_aw_avg,n_tiers=7)   #By monster

#TA
tl_ta_quest   = make_tiers(ta_aw_avg_q,n_tiers=7) #By quest
tl_ta_monster = make_tiers(ta_aw_avg,n_tiers=7)   #By monster

In [ ]:
#Display tier lists

print('Freestyle Tier list - Best times by MONSTER')
tl_fs_monster

In [ ]:
print('Freestyle Tier list - Best times by QUEST')
tl_fs_quest

In [ ]:
print('TA Rules Tier list - Best times by MONSTER')
tl_ta_monster

In [ ]:
print('TA Rules Tier list - Best times by QUEST')
tl_ta_quest

# 3. - REMOVE OUTLIERS, THEN RANK
Find out which speedrun times are outliers (too fast or too slow for a given weapon), remove these quests and then proceed with a tier list analysis, as done in section 2

## 3.1 - BOXPLOTS
Do some boxplots to quickly see the outliers

In [ ]:
#Freestyle runs
plt.figure(figsize=figure_size)
sns.boxplot(x='Weapon',y='Time (s)',data=freestyle).set_title("Freestyle runs",fontsize=font_size)

#TA runs
plt.figure(figsize=figure_size)
sns.boxplot(x='Weapon',y='Time (s)',data=freestyle,whis=1.5).set_title("Freestyle runs",fontsize=font_size)
# sns.boxplot(x='Weapon',y='Time (s)',data=ta).set_title("TA runs",fontsize=font_size)

In [ ]:
#Eliminate outliers
fs_out = remove_outliers(freestyle,mult=1.5) #Freestyle
ta_out = remove_outliers(ta,mult=1.5)

#Plot to check
plt.figure(figsize=figure_size)
sns.boxplot(x='Weapon',y='Time (s)',data=fs_out).set_title("Freestyle runs - No Outliers",fontsize=font_size)

plt.figure(figsize=figure_size)
sns.boxplot(x='Weapon',y='Time (s)',data=ta_out).set_title("TA runs - No Outliers",fontsize=font_size)

## 3.2 - BARPLOTS & TIME TABLES
Display bar plots of the average top clear times for each weapon, separated both by Monster and by quest.
Also display the average top clear times themselves.

Tier lists will be shown in the next section

In [ ]:
#Filter out so we have only QUESTS with at least one entry per weapon
fs_out_quest = filter_by_weapon(fs_out,filter_by='Monster Name')
ta_out_quest = filter_by_weapon(ta_out,filter_by='Monster Name')

#Get average times
fs_out_quest = average_top_runs(fs_out_quest,'Monster/Weapon')
ta_out_quest = average_top_runs(ta_out_quest,'Monster/Weapon')

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=fs_out_quest).set_title('Average TOP clear time by Monster - Freestyle', fontsize =font_size)

plt.figure(figsize=figure_size)
sns.barplot(x='Weapon',y='Time (s)',data=ta_out_quest).set_title('Average TOP clear time by Monster - TA', fontsize =font_size)

#Create 'Weapon (long)' column to have long form weapon names
#First, get the inverse weapon_dict to go from short names to long names
inv_weapon_dict = {value: key for key, value in weapon_dict.items()}

#Apply mapping from inv_weapon-dict
fs_out_quest['Weapon (long)']=fs_out_quest['Weapon'].apply(lambda x: inv_weapon_dict[x])
ta_out_quest['Weapon (long)']=ta_out_quest['Weapon'].apply(lambda x: inv_weapon_dict[x])

#Numerical values
print('Freestyle average top clear times per weapon (1 entry per monster)')
print(fs_aw_avg[['Weapon (long)','Time (s)']])
print('\nTA Rules average top clear times per weapon (1 entry per monster)')
print(ta_aw_avg[['Weapon (long)','Time (s)']])

## 3.3 - TIER LISTS

In [ ]:
#Make some tier lists

#Freestyle
tl_fs_out_quest   = make_tiers(fs_out_quest,n_tiers=7) #By quest

#TA
tl_ta_out_quest   = make_tiers(ta_out_quest,n_tiers=7) #By quest

In [ ]:
#Display tier lists

print('Freestyle Tier list - Best times by QUEST (No Outliers)')
tl_fs_out_quest

In [ ]:
print('TA Tier list - Best times by QUEST (No Outliers)')
tl_ta_out_quest